<a href="https://colab.research.google.com/github/TrinhDinhPhuc/CompoundClassification/blob/main/Compound_Classification_using_Graph_Convolutional_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  22229      0 --:--:-- --:--:-- --:--:-- 22229


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem

     |████████████████████████████████| 552kB 8.7MB/s 


In [3]:
import deepchem as dc
from rdkit import Chem
import numpy as np
import pandas as pd
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

import warnings
warnings.filterwarnings("ignore")

In [4]:
!wget https://raw.githubusercontent.com/TrinhDinhPhuc/CompoundClassification/main/cmpd.csv

--2021-02-20 10:22:42--  https://raw.githubusercontent.com/TrinhDinhPhuc/CompoundClassification/main/cmpd.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530832 (518K) [text/plain]
Saving to: ‘cmpd.csv’

cmpd.csv            100%[===================>] 518.39K  --.-KB/s    in 0.03s   

2021-02-20 10:22:42 (16.4 MB/s) - ‘cmpd.csv’ saved [530832/530832]



In [5]:
cmpd_df = pd.read_csv('cmpd.csv')
cmpd_df.head()

inchikey  ... activity
0  FNHKPVJBJVTLMP-UHFFFAOYSA-N  ...   active
1  CUDVHEFYRIWYQD-UHFFFAOYSA-N  ...   active
2  TTZSNFLLYPYKIL-UHFFFAOYSA-N  ...   active
3  UOVCGJXDGOGOCZ-UHFFFAOYSA-N  ...   active
4  CUIHSIWYWATEQL-UHFFFAOYSA-N  ...   active

[5 rows x 4 columns]

In [6]:
## Create a label column where activity.active = 1 otherwise = 0  
cmpd_df["label"] = None
for row in cmpd_df.itertuples():
  if row.activity == "active":
    cmpd_df.at[row.Index,'label'] = 1
  else:
    cmpd_df.at[row.Index,'label'] = 0

In [7]:
cmpd_df["smileToMol"] = None

import deepchem as dc
from rdkit import Chem
def smileToMol(smile):
  # smile = 'CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)c(F)c2)ccn1'
  molecules = []
  molecules.append(Chem.MolFromSmiles(smile))
  featurizer = dc.feat.graph_features.ConvMolFeaturizer()
  mol_object = featurizer.featurize(molecules)
  return mol_object

for row in cmpd_df.itertuples():
  cmpd_df.at[row.Index,'smileToMol'] = smileToMol(row.smiles)
cmpd_df.head()

inchikey  ...                                         smileToMol
0  FNHKPVJBJVTLMP-UHFFFAOYSA-N  ...  [<deepchem.feat.mol_graphs.ConvMol object at 0...
1  CUDVHEFYRIWYQD-UHFFFAOYSA-N  ...  [<deepchem.feat.mol_graphs.ConvMol object at 0...
2  TTZSNFLLYPYKIL-UHFFFAOYSA-N  ...  [<deepchem.feat.mol_graphs.ConvMol object at 0...
3  UOVCGJXDGOGOCZ-UHFFFAOYSA-N  ...  [<deepchem.feat.mol_graphs.ConvMol object at 0...
4  CUIHSIWYWATEQL-UHFFFAOYSA-N  ...  [<deepchem.feat.mol_graphs.ConvMol object at 0...

[5 rows x 6 columns]

In [8]:
data_train = cmpd_df[cmpd_df.group == "train"]
data_test = cmpd_df[cmpd_df.group == "test"]
data_train.to_csv("data_train.csv", index=False)
data_test.to_csv("data_test.csv", index=False)
print("\ndata_train.shape=",data_train.shape, "data_test.shape=", data_test.shape)


data_train.shape= (3977, 6) data_test.shape= (1553, 6)


In [9]:
3977 + 1553

5530

In [10]:
### notes: I combined data_train and data_test files into one.
### The first 3977 rows idx[0:3976] comes from the data_train and the rest of it belongs to the data_test 

In [11]:
## Download the final data that was saved
!wget https://github.com/TrinhDinhPhuc/CompoundClassification/blob/main/data.csv.gz?raw=true
!mv data.csv.gz?raw=true data.csv.gz
!mv data.csv.gz /tmp

--2021-02-20 10:23:04--  https://github.com/TrinhDinhPhuc/CompoundClassification/blob/main/data.csv.gz?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/TrinhDinhPhuc/CompoundClassification/raw/main/data.csv.gz [following]
--2021-02-20 10:23:04--  https://github.com/TrinhDinhPhuc/CompoundClassification/raw/main/data.csv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/TrinhDinhPhuc/CompoundClassification/main/data.csv.gz [following]
--2021-02-20 10:23:04--  https://raw.githubusercontent.com/TrinhDinhPhuc/CompoundClassification/main/data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

##### The SpecifiedSplitter Class written by Deepchem does not allow me to specify data_train, data_validation, and data_test by using slicing, therefore, I copied SpecifiedSplitter and Splitter classes from Deepchem's source and modified the SpecifiedSplitter class.


In [15]:
import inspect
import os
import random
import tempfile
import itertools
import logging
from typing import Any, Dict, List, Iterator, Optional, Sequence, Tuple

import numpy as np
import pandas as pd

import deepchem as dc
from deepchem.data import Dataset, DiskDataset
from deepchem.utils import get_print_threshold
logger = logging.getLogger(__name__)


class Splitter(object):
  """Splitters split up Datasets into pieces for training/validation/testing.
  In machine learning applications, it's often necessary to split up a dataset
  into training/validation/test sets. Or to k-fold split a dataset (that is,
  divide into k equal subsets) for cross-validation. The `Splitter` class is
  an abstract superclass for all splitters that captures the common API across
  splitter classes.
  Note that `Splitter` is an abstract superclass. You won't want to
  instantiate this class directly. Rather you will want to use a concrete
  subclass for your application.
  """

  def k_fold_split(self,
                   dataset: Dataset,
                   k: int,
                   directories: Optional[List[str]] = None,
                   **kwargs) -> List[Tuple[Dataset, Dataset]]:
    """
    Parameters
    ----------
    dataset: Dataset
      Dataset to do a k-fold split
    k: int
      Number of folds to split `dataset` into.
    directories: List[str], optional (default None)
      List of length 2*k filepaths to save the result disk-datasets.
    Returns
    -------
    List[Tuple[Dataset, Dataset]]
      List of length k tuples of (train, cv) where `train` and `cv` are both `Dataset`.
    """
    logger.info("Computing K-fold split")
    if directories is None:
      directories = [tempfile.mkdtemp() for _ in range(2 * k)]
    else:
      assert len(directories) == 2 * k
    cv_datasets = []
    train_ds_base = None
    train_datasets = []
    # rem_dataset is remaining portion of dataset
    if isinstance(dataset, DiskDataset):
      rem_dataset = dataset
    else:
      rem_dataset = DiskDataset.from_numpy(dataset.X, dataset.y, dataset.w,
                                           dataset.ids)

    for fold in range(k):
      # Note starts as 1/k since fold starts at 0. Ends at 1 since fold goes up
      # to k-1.
      frac_fold = 1. / (k - fold)
      train_dir, cv_dir = directories[2 * fold], directories[2 * fold + 1]
      fold_inds, rem_inds, _ = self.split(
          rem_dataset,
          frac_train=frac_fold,
          frac_valid=1 - frac_fold,
          frac_test=0,
          **kwargs)
      cv_dataset = rem_dataset.select(fold_inds, select_dir=cv_dir)
      cv_datasets.append(cv_dataset)
      # FIXME: Incompatible types in assignment (expression has type "Dataset", variable has type "DiskDataset")
      rem_dataset = rem_dataset.select(rem_inds)  # type: ignore

      train_ds_to_merge: Iterator[Dataset] = filter(
          None, [train_ds_base, rem_dataset])
      train_ds_to_merge = filter(lambda x: len(x) > 0, train_ds_to_merge)
      train_dataset = DiskDataset.merge(train_ds_to_merge, merge_dir=train_dir)
      train_datasets.append(train_dataset)

      update_train_base_merge: Iterator[Dataset] = filter(
          None, [train_ds_base, cv_dataset])
      train_ds_base = DiskDataset.merge(update_train_base_merge)
    return list(zip(train_datasets, cv_datasets))

  def train_valid_test_split(self,
                             dataset: Dataset,
                             train_dir: Optional[str] = None,
                             valid_dir: Optional[str] = None,
                             test_dir: Optional[str] = None,
                             frac_train: float = 0.8,
                             frac_valid: float = 0.1,
                             frac_test: float = 0.1,
                             seed: Optional[int] = None,
                             log_every_n: int = 1000,
                             **kwargs) -> Tuple[Dataset, Dataset, Dataset]:
    """ Splits self into train/validation/test sets.
    Returns Dataset objects for train, valid, test.
    Parameters
    ----------
    dataset: Dataset
      Dataset to be split.
    train_dir: str, optional (default None)
      If specified, the directory in which the generated
      training dataset should be stored. This is only
      considered if `isinstance(dataset, dc.data.DiskDataset)`
    valid_dir: str, optional (default None)
      If specified, the directory in which the generated
      valid dataset should be stored. This is only
      considered if `isinstance(dataset, dc.data.DiskDataset)`
      is True.
    test_dir: str, optional (default None)
      If specified, the directory in which the generated
      test dataset should be stored. This is only
      considered if `isinstance(dataset, dc.data.DiskDataset)`
      is True.
    frac_train: float, optional (default 0.8)
      The fraction of data to be used for the training split.
    frac_valid: float, optional (default 0.1)
      The fraction of data to be used for the validation split.
    frac_test: float, optional (default 0.1)
      The fraction of data to be used for the test split.
    seed: int, optional (default None)
      Random seed to use.
    log_every_n: int, optional (default 1000)
      Controls the logger by dictating how often logger outputs
      will be produced.
    Returns
    -------
    Tuple[Dataset, Optional[Dataset], Dataset]
      A tuple of train, valid and test datasets as dc.data.Dataset objects.
    """
    logger.info("Computing train/valid/test indices")
    train_inds, valid_inds, test_inds = self.split(
        dataset,
        seed=seed,
        log_every_n=log_every_n)
    if train_dir is None:
      train_dir = tempfile.mkdtemp()
    if valid_dir is None:
      valid_dir = tempfile.mkdtemp()
    if test_dir is None:
      test_dir = tempfile.mkdtemp()
    train_dataset = dataset.select(train_inds, train_dir)
    valid_dataset = dataset.select(valid_inds, valid_dir)
    test_dataset = dataset.select(test_inds, test_dir)
    if isinstance(train_dataset, DiskDataset):
      train_dataset.memory_cache_size = 40 * (1 << 20)  # 40 MB

    return train_dataset, valid_dataset, test_dataset

  def train_test_split(self,
                       dataset: Dataset,
                       train_dir: Optional[str] = None,
                       test_dir: Optional[str] = None,
                       frac_train: float = 0.8,
                       seed: Optional[int] = None,
                       **kwargs) -> Tuple[Dataset, Dataset]:
    """Splits self into train/test sets.
    Returns Dataset objects for train/test.
    Parameters
    ----------
    dataset: data like object
      Dataset to be split.
    train_dir: str, optional (default None)
      If specified, the directory in which the generated
      training dataset should be stored. This is only
      considered if `isinstance(dataset, dc.data.DiskDataset)`
      is True.
    test_dir: str, optional (default None)
      If specified, the directory in which the generated
      test dataset should be stored. This is only
      considered if `isinstance(dataset, dc.data.DiskDataset)`
      is True.
    frac_train: float, optional (default 0.8)
      The fraction of data to be used for the training split.
    seed: int, optional (default None)
      Random seed to use.
    Returns
    -------
    Tuple[Dataset, Dataset]
      A tuple of train and test datasets as dc.data.Dataset objects.
    """
    valid_dir = tempfile.mkdtemp()
    train_dataset, _, test_dataset = self.train_valid_test_split(
        dataset,
        train_dir,
        valid_dir,
        test_dir,
        frac_train=frac_train,
        frac_test=1 - frac_train,
        frac_valid=0.,
        seed=seed,
        **kwargs)
    return train_dataset, test_dataset

  def split(self,
            dataset: Dataset,
            frac_train: float = 0.8,
            frac_valid: float = 0.1,
            frac_test: float = 0.1,
            seed: Optional[int] = None,
            log_every_n: Optional[int] = None) -> Tuple:
    """Return indices for specified split
    Parameters
    ----------
    dataset: dc.data.Dataset
      Dataset to be split.
    seed: int, optional (default None)
      Random seed to use.
    frac_train: float, optional (default 0.8)
      The fraction of data to be used for the training split.
    frac_valid: float, optional (default 0.1)
      The fraction of data to be used for the validation split.
    frac_test: float, optional (default 0.1)
      The fraction of data to be used for the test split.
    log_every_n: int, optional (default None)
      Controls the logger by dictating how often logger outputs
      will be produced.
    Returns
    -------
    Tuple
      A tuple `(train_inds, valid_inds, test_inds)` of the indices (integers) for
      the various splits.
    """
    raise NotImplementedError

  def __str__(self) -> str:
    """Convert self to str representation.
    Returns
    -------
    str
      The string represents the class.
    Examples
    --------
    >>> import deepchem as dc
    >>> str(dc.splits.RandomSplitter())
    'RandomSplitter'
    """
    args_spec = inspect.getfullargspec(self.__init__)  # type: ignore
    args_names = [arg for arg in args_spec.args if arg != 'self']
    args_num = len(args_names)
    args_default_values = [None for _ in range(args_num)]
    if args_spec.defaults is not None:
      defaults = list(args_spec.defaults)
      args_default_values[-len(defaults):] = defaults

    override_args_info = ''
    for arg_name, default in zip(args_names, args_default_values):
      if arg_name in self.__dict__:
        arg_value = self.__dict__[arg_name]
        # validation
        # skip list
        if isinstance(arg_value, list):
          continue
        if isinstance(arg_value, str):
          # skip path string
          if "\\/." in arg_value or "/" in arg_value or '.' in arg_value:
            continue
        # main logic
        if default != arg_value:
          override_args_info += '_' + arg_name + '_' + str(arg_value)
    return self.__class__.__name__ + override_args_info

  def __repr__(self) -> str:
    """Convert self to repr representation.
    Returns
    -------
    str
      The string represents the class.
    Examples
    --------
    >>> import deepchem as dc
    >>> dc.splits.RandomSplitter()
    RandomSplitter[]
    """
    args_spec = inspect.getfullargspec(self.__init__)  # type: ignore
    args_names = [arg for arg in args_spec.args if arg != 'self']
    args_info = ''
    for arg_name in args_names:
      value = self.__dict__[arg_name]
      # for str
      if isinstance(value, str):
        value = "'" + value + "'"
      # for list
      if isinstance(value, list):
        threshold = get_print_threshold()
        value = np.array2string(np.array(value), threshold=threshold)
      args_info += arg_name + '=' + str(value) + ', '
    return self.__class__.__name__ + '[' + args_info[:-2] + ']'



### *******************************************************
### -------------------------------------------------------
### -------------------------------------------------------
### -------------------------------------------------------
### I did modify the following class to split the dataset by index

class SpecifiedSplitter(Splitter):
  def __init__(self,
              #  train_indices = None,
               valid_indices = list(range(3001, 3977)), # 3001 -> 3976
               test_indices  = list(range(3977, 5530))): # 3977:

    self.valid_indices = valid_indices
    self.test_indices = test_indices
  def split(self,
            dataset: Dataset,
            seed: Optional[int] = None,
            log_every_n: Optional[int] = None
           ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    train_indices = list(range(0, 3001)) # 0 -> 3000
    return (np.array(train_indices), np.array(self.valid_indices),
            np.array(self.test_indices))

In [16]:
import os
import deepchem as dc
from deepchem.molnet.load_function.molnet_loader import TransformerGenerator, _MolnetLoader
from deepchem.data import Dataset
from typing import List, Optional, Tuple, Union

# from deepchem.splits.splitters import SpecifiedSplitter

TASKS = [
    'active', 'inactive'
]

class _Loader(_MolnetLoader):
  def create_dataset(self) -> Dataset:
    dataset_file = os.path.join(self.data_dir, "data.csv.gz")
    if not os.path.exists(dataset_file):
      print("The file does not exist !!!")
    loader = dc.data.CSVLoader(
        tasks=self.tasks, feature_field="smiles", featurizer=self.featurizer)
    return loader.create_dataset(dataset_file, shard_size=8192)

def load_dataset(
    featurizer: Union[dc.feat.Featurizer, str] = 'ECFP',
    splitter = SpecifiedSplitter(),
    transformers: List[Union[TransformerGenerator, str]] = ['balancing'],
    reload: bool = True,
    data_dir: Optional[str] = None,
    save_dir: Optional[str] = None,
    **kwargs

) -> Tuple[List[str], Tuple[Dataset, ...], List[dc.trans.Transformer]]:

  loader = _Loader(featurizer, splitter, transformers, TASKS,
                        data_dir, save_dir, **kwargs)
  return loader.load_dataset('data_train', reload)

In [17]:
import deepchem as dc
tasks, datasets, transformers = load_dataset(featurizer='GraphConv')

train_dataset, valid_dataset, test_dataset = datasets

In [18]:
print(train_dataset.X.shape[0], valid_dataset.X.shape[0], test_dataset.X.shape[0])
''' 
- 3001 rows for training
- 976 rows for validation
- 1553 rows for testing
'''

3001 976 1553


' \n- 3001 rows for training\n- 976 rows for validation\n- 1553 rows for testing\n'

In [19]:
train_dataset.X.shape[0] + valid_dataset.X.shape[0] + test_dataset.X.shape[0]
## it must return 5530 to be correct

5530

In [22]:
## I use a simple Graph Convolutional Model provided by deepchem to do the classification
## https://deepchem.readthedocs.io/en/latest/api_reference/models.html#graphconvmodel
model = dc.models.GraphConvModel(2, # n_tasks=2  
                                 mode='classification', 
                                 dropout = 0.25, ## we are facing overfiting, thus I set dropout = 0.25 to tackle the problem
                                 dense_layer_size =128, # dense_layer_size =128 as default
                                 tensorboard=True,
                                 model_dir = "/tmp/tensorboard")
model.fit(train_dataset, nb_epoch=200)
## Note: the training process will take around 1 to 2 mins only

0.4023862838745117

In [23]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('Training set score:', model.evaluate(train_dataset, [metric], transformers))
print('Test set score:', model.evaluate(test_dataset, [metric], transformers))

Training set score: {'roc_auc_score': 0.9875792243743441}
Test set score: {'roc_auc_score': 0.925342893340433}


## The results will be something like these:
#### Training set score: {'roc_auc_score': 0.9899005978872861}
#### Test set score: {'roc_auc_score': 0.9482054330462883}

### Please ignore the following cells, They are just my customized model that I want to play around with.

In [26]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100
n_tasks = 2

class MyGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(MyGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

In [27]:
model = dc.models.KerasModel(
    MyGraphConvModel(), 
    loss=dc.models.losses.CategoricalCrossEntropy())

In [28]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import numpy as np

def data_generator(dataset, epochs=1):
  for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
    multiConvMol = ConvMol.agglomerate_mols(X_b)
    inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
    labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
    weights = [w_b]
    yield (inputs, labels, weights)

In [29]:
model.fit_generator(data_generator(train_dataset, epochs=200))

0.11460309028625489

In [30]:
print('Training set score:', model.evaluate_generator(data_generator(train_dataset), [metric], transformers))
print('Test set score:', model.evaluate_generator(data_generator(test_dataset), [metric], transformers))

Training set score: {'roc_auc_score': 0.7089584658068913}
Test set score: {'roc_auc_score': 0.6752864822989295}
